In [1]:
import torch.utils.data as data
from chess_dataset import ChessDataset

dataset = ChessDataset("tests/test.engine.csv")
data_loader = data.DataLoader(dataset, batch_size=4, shuffle=True)

In [2]:
import torch
import models

m = models.create('Rollout.v0')
m.cuda()

Policy(
  (conv1): Sequential(
    (0): Conv2d (23, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
  )
  (hidden_conv_layers): Sequential(
    (0): Sequential(
      (0): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (1): Sequential(
      (0): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (2): Sequential(
      (0): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (3): Sequential(
      (0): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (4): Sequential(
      (0): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
  )
  (final_conv_layer): Sequential(
    (0): Conv2d (128, 73, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
  )
)

In [3]:
import torch.optim as optim
import torch. nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(m.parameters(), lr=0.001, momentum=0.9, nesterov=True)

In [4]:
%%timeit -n 1 -r 10

from torch.autograd import Variable

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        if torch.cuda.is_available():
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = m(inputs)
        outputs = outputs.view(outputs.shape[0], -1)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
print('Training done')

Training done
Training done
Training done
Training done
Training done
Training done
Training done
Training done
Training done
Training done
567 ms ± 210 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [5]:
%%timeit

from torch.autograd import Variable
import torch

m.eval()
inputs = Variable(torch.randn(1, 23, 8, 8).cuda(), volatile=True)
outputs = m(inputs)

1.35 ms ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
